# Dicitonary Functionals

Provides functionals for diciontary related stuff. In most cases, named tuples are also supported.

I'm not too sure about the jargon here: I'm using _functional_ to mean a function that returns a function. Typically, a functional's parameters determine the parameter of the function that's returned.

In [ ]:
"""Dictionary functionals contains functions that return dictionary manipulating functions.

Functionals take zero or more parameters that specify how to manipulate a dictionary. Their returned 
functions take one or more dictionaries and return a value.

Functionals
-----------
has_keys:
    Tests whether dictionary has the keys in a list.

filter_values:
    Tests whether values in a dictionary satisfy a predicate.

extract_keys:
    Return a dictionary including only certain keys.

drop_keys:
    Return a dictionary excluding certain keys.

map_values:
    Apply a mapping to specified keys, returning the transformed dictionary.

flatten_dictionary:
    Return a dictionary that inherits the key-value pairs of any dictionary-valued key.

"""

In [ ]:
from typing import List, Callable, Dict, Any
from functools import singledispatch, lru_cache
from collections import namedtuple

In [ ]:
FunctionalMap = Dict[Any, Callable]

In [ ]:
def has_keys(keys: List, every: bool = True, only: bool = False) -> Callable[[dict], dict]:
    """True if all/only keys are dictionary.
    
    every: bool
        Requires all keys to be in a dictionary.

    only: bool
        Requires all dictionary keys to be in keys.
    
    If both all and only are false, returns True if and only if the dictionary is not
    empty.

    """
    def _func(dict_):
        if every and only:
            has_all = all([k in keys for k in dict_])
            has_only = all([k in dict_.keys() for k in keys])
            return has_all and has_only
        if every:
            return all([k in dict_ for k in keys])
        if only:
            return all([k in keys for k in dict_])
        return bool(dict_)
    return _func


In [ ]:
def filter_values(mapping: FunctionalMap):
    """Return True if predicate in mapping is true for all.""" 
    @singledispatch
    def _func(dict_: dict) -> Callable[[Dict[Any, Callable]], dict]:
        return all(mapping[k](dict_[k]) for k in dict_)

    @_func.register
    def _(dict_: tuple):
        return all(mapping[k](getattr(dict_, k)) for k in mapping)
    return _func

test_nt = namedtuple('test_nt', ['a', 'b'])
d = {'a': lambda x: x > 2}
assert not filter_values(d)({'a': 1})
assert filter_values(d)({'a': 3})
assert filter_values(d)(test_nt(3, 1))


In [ ]:
list_of_dicts = [
    {'a': 1, 'b': 2},
    {'c': 2, 'b': 3},
    {'a': 2, 'b': 3, 'c': 0}
]

has_keys_test = list(filter(has_keys(['b'], every=False), list_of_dicts))
print(has_keys_test)
assert len(has_keys_test) == 3
assert all([('b' in dict_) for dict_ in has_keys_test])

In [ ]:
def extract_keys(keys: List[str]) -> Callable[[dict], dict]:
    """Returns dictionary whose only keys are keys parameter."""
    @singledispatch
    def _func(dict_: dict) -> dict:
        return {k: v for k, v in dict_.items() if k in keys}

    @_func.register
    def _(dict_: tuple):
        fields = [field for field in dict_.fieldnames if field in keys]
        new_namedtuple = nt_builder('extracted', *fields)
        return new_namedtuple._make(getattr(dict_, f) for f in fields)

    return _func

def drop_keys(keys: List[str]) -> Callable[[dict], dict]:
    """Returns dictionary without keys in keys parameter."""
    @singledispatch
    def _func(dict_: dict) -> dict:
        return {k: v for k, v in dict_.items() if k not in keys}

    @_func.register
    def _(dict_: tuple):
        fields = [field for field in dict_.fieldnames if field not in keys]
        new_namedtuple = nt_builder('dropped', *fields)
        return new_namedtuple._make(getattr(dict_, f) for f in fields)

    return _func


In [ ]:
def map_values(mapping: FunctionalMap):
    """Apply a mapping to each column."""
    @singledispatch
    def _func(dict_: dict) -> dict:
        anon = (lambda k, v: mapping[k](v) if k in mapping else v)
        return {k: anon(k, v) for k, v in dict_.items()}

    @_func.register
    def _(dict_: tuple) -> tuple:
        mapped = {k: mapping[k](getattr(dict_, k)) for k in mapping}
        return dict_._replace(**mapped)
    
    return _func


In [ ]:
%timeit map_values({'a': lambda x: x**2})(test_nt(3, 1))
#isinstance(test_nt(3, 1)       , tuple)
        


In [ ]:
%timeit map_values({'a': lambda x: x**2})({'a': 3, 'b': 1, 'c': 2})

In [8]:
from functionals.dict_functionals import flatten_dict
assert flatten_dict()({"a": 1, 'd': {"b":1, 'c': 1}}) == {'a': 1, 'd__b': 1, 'd__c': 1}

{'a': 1, 'd__b': 1, 'd__c': 1}

In [7]:

# UnboundLocalError in Python
var = 20 # a is global variable

def my_function():
    print(var)
    var = 'Hello' # This causes UnboundLocalError
    print(var)

my_function() # Calling function


UnboundLocalError: local variable 'var' referenced before assignment

In [ ]:
import math
def sequential_func(*functions):
    """Apply functions in order."""
    def _func(*args):
        for function in functions:
            try:
                args = function(*args)
            except TypeError:
                args = function(args)
        return args
    return _func


In [ ]:
def explode_dict(key):
    """Return an iterable of dictionaries from a dictionary with iterable keys.

    Parameters
    ----------
    key
        The key to explode.

    Returns
    -------
        A function that takes a dictionary and returns an iterable of
        dictionaries exploded by the key.

    Example
    -------
        list(explode_dict(a)({a: [1, 2], b: 3}))
        >>>[{a: 1, b: 3}, {a: 2, b: 3}]
    """

    def func_(dict_):
        for v in dict_[key]:
            out = {k: v for k, v in dict_.items()}
            out[key] = v
            yield out
    return func_

assert list(explode_dict('a')({'a': [1, 2], 'b': 3})) == [{'a': 1, 'b': 3}, {'a': 2, 'b': 3}]